In [1]:
from nba_api.stats.static import players
from nba_api.stats.endpoints import shotchartdetail
from nba_api.stats.static import teams

In [2]:
import os
import pandas as pd
import requests
import numpy as np
import json

### Player Info

In [5]:
# get all player ids
player_dict = players.get_players()

In [50]:
lebron = [player for player in player_dict if
         player['full_name']== 'LeBron James']

kd = [player for player in player_dict if
         player['full_name']== 'Kevin Durant']

curry = [player for player in player_dict if
         player['full_name']== 'Stephen Curry']

In [51]:
lebron,kd,curry = lebron[0], kd[0], curry[0]

In [52]:
lebron_id, kd_id, curry_id = lebron['id'], kd['id'], curry['id']
# id are unique to players across all seasons

In [9]:
teams_list = teams.get_teams()

what teams have they been on:

lebron (cleveland, maimi, lakers)

kd(okc, golden state, nets)

steph(golden state)

In [43]:
gsw = [team for team in teams_list if
               team['abbreviation']== 'GSW']

In [44]:
gsw

[{'id': 1610612744,
  'full_name': 'Golden State Warriors',
  'abbreviation': 'GSW',
  'nickname': 'Warriors',
  'city': 'Golden State',
  'state': 'California',
  'year_founded': 1946}]

In [53]:
okc = [team for team in teams_list if
               team['abbreviation']== 'OKC']

miami = [team for team in teams_list if
               team['abbreviation']== 'MIA']

cleveland = [team for team in teams_list if
               team['abbreviation']== 'CLE']

lakers = [team for team in teams_list if
               team['abbreviation']== 'LAL']

nets = [team for team in teams_list if
               team['abbreviation']== 'BKN']


In [55]:
gsw,okc,miami,cleveland,lakers,nets = gsw[0],okc[0],miami[0],cleveland[0],lakers[0],nets[0] 

In [56]:
gsw_id,okc_id,miami_id,cleveland_id,lakers_id,nets_id = gsw['id'],okc['id'],miami['id'],cleveland['id'],lakers['id'],nets['id']

In [17]:
headers = ['GRID_TYPE','GAME_ID','GAME_EVENT_ID','PLAYER_ID','PLAYER_NAME',
 'TEAM_ID','TEAM_NAME','PERIOD','MINUTES_REMAINING','SECONDS_REMAINING',
 'EVENT_TYPE','ACTION_TYPE','SHOT_TYPE','SHOT_ZONE_BASIC','SHOT_ZONE_AREA',
 'SHOT_ZONE_RANGE','SHOT_DISTANCE','LOC_X','LOC_Y',
 'SHOT_ATTEMPTED_FLAG','SHOT_MADE_FLAG','GAME_DATE','HTM','VTM']

In [18]:
# for years lets do 2010-11 as the starting year, so we can
# start with the first lebron year in miami
# end with 2019-20

In [18]:
years = ['2010-11','2011-12','2012-13','2013-14','2014-15','2015-16',
        '2016-17','2017-18','2018-19','2019-20']

### write functions for api access

In [22]:
new_json = shotchartdetail.ShotChartDetail(player_id = curry_id, 
                                             season_type_all_star = 'Regular Season',
                                             context_measure_simple = 'FGA',
                                             team_id = gsw_id,
                                             season_nullable = '2010-11')

In [23]:
def add_data(input, frame):
    # input = json, data wer're adding
    # frame = pd.df, dataframe I'm adding to 
    data = json.loads(input.get_json())
    rel_data = data['resultSets'][0]
    rows = rel_data['rowSet']
    headers = rel_data['headers']
    temp_df = pd.DataFrame(rows)
    temp_df.columns = headers
    frame = pd.concat([frame,temp_df],axis=0)
    return frame


In [24]:
def get_seasons(years,p_id,t_id,frame,season_type='Regular Season',context='FGA'):
    # years = list of str, season we get data from
    # p_id = int, id of player
    # t_id = int, id of team
    # season_type = str, regular post or allstar
    # context = str, what kind of data
    # frame = pd.df, dataframe to add info to
    for year in years:
        new_json = shotchartdetail.ShotChartDetail(player_id = p_id, 
                                             season_type_all_star = season_type,
                                             context_measure_simple = context,
                                             team_id = t_id,
                                             season_nullable = year)
        frame = add_data(new_json,frame)
        
    return frame
    

### Get Curry Data

In [19]:
curry_years = years[:]

In [20]:
curry_df = pd.DataFrame(columns = headers)

In [25]:
curry_df = get_seasons(curry_years,curry_id,gsw_id,curry_df)

In [74]:
# curry_df.to_csv('C:\\Users\\kamka\\curry_data.csv')

### Get Lebron Data

In [27]:
lebron_years_miami = years[:4]
lebron_years_cleveland = years[4:8]
lebron_years_la = years[8:10]

In [29]:
lebron_df = pd.DataFrame(columns=headers)

In [ ]:
# need to access data in multiple steps because of free api restrictions

In [31]:
# start with the miami years
lebron_df = get_seasons(lebron_years_miami,lebron_id,miami_id,lebron_df)

In [54]:
# then get second cleveland
lebron_df = get_seasons(lebron_years_cleveland,lebron_id,cleveland_id,lebron_df)

In [58]:
# end with lakers years
lebron_df = get_seasons(lebron_years_la,lebron_id,lakers_id,lebron_df)

In [73]:
# lebron_df.to_csv('C:\\Users\\kamka\\lebron_data.csv')

### Get KD Data

In [64]:
kd_years_okc,kd_years_gsw,kd_years_nets = years[:6],years[6:9],years[9]

In [67]:
kd_df = pd.DataFrame(columns=headers)

In [68]:
# okc years
kd_df = get_seasons(kd_years_okc,kd_id,okc_id,kd_df)

In [69]:
# gsw years
kd_df = get_seasons(kd_years_gsw,kd_id,gsw_id,kd_df)

KD didn't play in the 2019-2020 nba season for the nets.

In [71]:
# kd_df.to_csv('C:\\Users\\kamka\\kd_data.csv')